## Bucles avanzados

La capacidad de sólo realizar bucles de R va mucho más allá de los bucles  estándar mencionados anteriormente. R como lenguaje de programación, proporciona la posibilidad de aplicar funciones a cada elemento de un vector, lista o matriz, de manera que se pueda escribir código pseudo-vectorizado, donde la vectorización normal no es posible. Otros bucles avanazados permiten realizar operaciones estadísticas con los datos.

### Replicación 

La función `rep` repite el argumento de entrada varias veces. Otra función relacionada, `replicate`, llama a una expresión varias veces. Ambas funciones, hacen exactamente lo mismo. La diferencia ocurre cuando la generación de números aleatorios está involucrada. Si se tiene por un momento que la función de generación de números aleatorios uniformes, `runif`, no está vectorizada, `rep` repetirá el mismo número aleatorio varias veces, pero `replicate` proporciona  un número diferente cada vez:

In [1]:
rep(runif(1), 5)

[1] 0.3155061 0.3155061 0.3155061 0.3155061 0.3155061

In [2]:
replicate(5, runif(1))

[1] 0.7869507 0.4655920 0.5094612 0.7608770 0.6473302

`replicate` se utiliza en  ejemplos más complicados: su uso principal es en el análisis de Monte Carlo, donde se repite un análisis  un número conocido de veces, y cada iteración es independiente de los demás.

In [3]:
help(replicate)

Este siguiente ejemplo estima que el tiempo de una persona para ir al trabajo a través de diferentes métodos de transporte y es  cuando la replicación es más útil.

La función `tiempo_para_viajar` utiliza `sample `para seleccionar aleatoriamente un modo de transporte (coche, autobús, tren o bicicleta), luego usa `rnorm` o `rlnorm` para encontrar un tiempo de viaje distribuido normalmente o lognormal(con parámetros que dependen del modo de transporte):

In [4]:
tiempo_para_viajar <- function()
{
    modo_de_transporte <- sample(
        c("car", "bus", "tren", "bicicleta"),
        size = 1,
        prob = c(0.1, 0.2, 0.3, 0.4)
)
#Encontramos el tiempo de viaje, dependiendo del transporte 
    tiempo <- switch(
        modo_de_transporte,
        car = rlnorm(1, log(30), 0.5),
        bus = rlnorm(1, log(40), 0.5),
        tren = rnorm(1, 30, 10),
        bicicleta = rnorm(1, 60, 5)
)
    names(tiempo) <- modo_de_transporte
    tiempo
}

La presencia de una sentencia `switch` hace que esta función sea muy difícil de vectorizar. Esto significa que para encontrar la distribución de los tiempos de desplazamiento, necesitamos llamar repetidamente a  `tiempo_para_viajar` para generar datos  cada día, la función `replicate` nos da la vectorización instantánea:

In [5]:
replicate(5, tiempo_para_viajar())

bicicleta       bus bicicleta      tren       bus 
 70.43425  74.41881  57.63483  26.80670  39.97614

Lanzamos una moneda `10` veces y queremos saber la probabilidad de conseguir más de 3 caras. Ahora bien, este es un problema trivial para la distribución binomial, pero supongamos que nos hemos olvidado de como hacerlo. Podemos resolver fácilmente este problema con una simulación de Monte Carlo. Utilizaremos el truco común de representar los sellos  con `0` y las caras con `1`, luego simularemos `10` lanzamientos de monedas `100.000` veces y veremos la frecuencia con la que esto sucede:

In [6]:
tiradas <- 100000
# experimento1  simula una sola ronda de lanzamiento de 10 monedas
# y retorna true  si el numero e caras es > 3

experimento1 <- function(){
  sum(sample(c(0,1),10,replace=T)) > 3
}

#ahora repetimos este experimento  '100000' veces.
mc.binom <- sum(replicate(tiradas,experimento1()))/tiradas
mc.binom

[1] 0.82895

In [7]:
# Comparamos con la funcion de distribución binomial de R

pbinom(3,10,0.5,lower.tail=FALSE)

[1] 0.828125

### La familia de funciones  `apply`


Gran cantidad de R está vectorizado, de hecho, su posición predeterminada debe ser de escribir código vectorizado. A menudo es más limpio para leer, e invariablemente le da beneficios de rendimiento en comparación con lod  bucle. En algunos casos, sin embargo, tratar de lograr la vectorización significa contorsionar su código de manera antinatural. En esos casos, la aplicación de la familia de funciones `apply` puede proporcionar la buscada vectorización.

#### La función lapply

El miembro de la familia `apply ` más simple y comúnmente utilizado es `lapply`, un  abreviado para `list apply` . La función  `lapply` toma una lista y una función como entradas, aplica la función a cada elemento de la lista  y devuelve otra lista de resultados. 

In [8]:
xL <- 1:5
xL

# retorna una lista
lapply(xL, is.integer)

[1] 1 2 3 4 5

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

In [9]:
# Usamos for para crear una lista independiente, de vectores
# normalmente distribuidos

len <- c(3, 4, 5)

x <- list()

for (i in 1:3) {
    x[[i]] <- rnorm(len[i])
    }
x

[[1]]
[1] 0.5981659 0.4590969 2.4221208

[[2]]
[1]  0.6015934 -0.1556072  0.1184133  0.2053498

[[3]]
[1] -1.4416748  0.5026391  0.8913344 -1.4906336 -0.1875493

In [10]:
# Con lappy

lapply(len, rnorm)

[[1]]
[1]  0.3486097 -0.9260349  0.4797771

[[2]]
[1] -1.5582715 -1.2097896 -0.7683578 -1.1026271

[[3]]
[1] -1.6731895  0.8157869  0.1766684  1.0622046  0.8195112

La versión del `lapply` es mucho más simple. Aplica `rnorm()` sobre  cada elemento en `len` y coloca cada resultado en una lista. La función `lapply()` es el bloque de construcción para muchos otros funcionales, por lo que es importante entender cómo funciona. He aquí una representación gráfica:

![](lapply.png)

`lapply()` está escrito en C por rendimiento, pero podemos crear una implementación R simple que haga lo mismo:

In [11]:
lapply2 <- function(x, f, ...) {
    out <- vector("list", length(x))
    for (i in seq_along(x)) {
        out[[i]] <- f(x[[i]], ...)
  }
  out
}

A partir de este código, `lapply()` es un contenedor para un patrón de bucle común: crea un contenedor para salida, aplica `f()` a cada componente de una lista y llena  el contenedor con los resultados. Todos los otros funcionales de la familia   son variaciones en este tema: utilizan simplemente diversos tipos de entrada o de salida. Los miembros de la familia `apply` se llaman **funciones de  orden superior**.



Desde que los data frames son listas, `lapply` también es útil cuando quieres hacer algo a cada columna de un data frame de datos:

In [12]:
#unlist convierte la salida de una lista a un vector
unlist(lapply(mtcars, class))

mpg       cyl      disp        hp      drat        wt      qsec        vs 
"numeric" "numeric" "numeric" "numeric" "numeric" "numeric" "numeric" "numeric" 
       am      gear      carb 
"numeric" "numeric" "numeric"

In [13]:
# Divide cada columna por la media
mtcars1<- lapply(mtcars, function(x) x / mean(x))

Las partes de `x` siempre se suministran como el primer argumento de `f`. Si se desea variar un argumento diferente, se puede utilizar una función anónima. El siguiente ejemplo varía la cantidad de partes  aplicado al calcular la media de un objeto `x` fijo.

In [14]:
trims <- c(0, 0.1, 0.2, 0.5)
x <- rcauchy(1000)
unlist(lapply(trims, function(trim) mean(x, trim = trim)))

[1] 0.24792162 0.11435191 0.06526379 0.04095040

#### La función sapply

La lista no siempre es un contenedor favorable para los resultados. A veces, queremos que se pongan en un simple vector o en una matriz. La función `sapply` una abreviación de `simplifying list apply`, simplifica el resultado según su estructura. Supongamos que aplicamos el cuadrado a cada elemento de `1:10`. Si lo hacemos con `lapply`, tendremos una lista de números cuadrados. Este resultado parece un poco pesado y redundante porque la lista resultante es en realidad una lista de vectores numéricos de un solo valor. Sin embargo, quizás queramos mantener los resultados como un vector:

In [15]:
sapply(xL, is.integer)

[1] TRUE TRUE TRUE TRUE TRUE

In [16]:
sapply(1:10, function(i) i ^ 2)

[1]   1   4   9  16  25  36  49  64  81 100

Si la función de aplicación devuelve un vector multi-elemento cada vez, `sapply` pondrá los resultados en una matriz en la que cada vector devuelto ocupa una columna:

In [17]:
sapply(1:10, function(i) c(i, i ^ 2))

1,2,3,4,5,6,7,8,9,10
1,4,9,16,25,36,49,64,81,100


In [18]:
# Ejemplo

primo_factores <- list(
    dos = 2,
    tres = 3,
    cuatro = c(2, 2),
    cinco = 5,
    seis= c(2, 3),
    siete = 7,
    ocho = c(2, 2, 2),
    nueve = c(3, 3),
    diez = c(2, 5)
)
head(primo_factores)

$dos
[1] 2

$tres
[1] 3

$cuatro
[1] 2 2

$cinco
[1] 5

$seis
[1] 2 3

$siete
[1] 7

Tratar de encontrar el valor único en cada elemento de lista es difícil de hacer de una manera vectorizada. Podríamos escribir un bucle para examinar cada elemento:

In [19]:
unico_primos <- vector("list", length(primo_factores))
for(i in seq_along(primo_factores))
{
    unico_primos[[i]] <- unique(primo_factores[[i]])
}
names(unico_primos) <- names(primo_factores)
unico_primos

$dos
[1] 2

$tres
[1] 3

$cuatro
[1] 2

$cinco
[1] 5

$seis
[1] 2 3

$siete
[1] 7

$ocho
[1] 2

$nueve
[1] 3

$diez
[1] 2 5

In [20]:
sapply(primo_factores, unique) # retorna una lista

$dos
[1] 2

$tres
[1] 3

$cuatro
[1] 2

$cinco
[1] 5

$seis
[1] 2 3

$siete
[1] 7

$ocho
[1] 2

$nueve
[1] 3

$diez
[1] 2 5

In [21]:
sapply(primo_factores, length) # retorna un vector

dos   tres cuatro  cinco   seis  siete   ocho  nueve   diez 
     1      1      2      1      2      1      3      2      2

In [22]:
sapply(primo_factores, summary) #returna una matriz

,dos,tres,cuatro,cinco,seis,siete,ocho,nueve,diez
Min.,2,3,2,5,2.00,7,2,3,2.00
1st Qu.,2,3,2,5,2.25,7,2,3,2.75
Median,2,3,2,5,2.50,7,2,3,3.50
Mean,2,3,2,5,2.50,7,2,3,3.50
3rd Qu.,2,3,2,5,2.75,7,2,3,4.25
Max.,2,3,2,5,3.00,7,2,3,5.00


Para el uso interactivo, esto es maravilloso porque usualmente obtiene automáticamente el resultado en la forma en que desea. Esta función requiere cierto cuidado si no está seguro de cuáles son sus entradas, aunque el resultado es a veces una lista y a veces un vector. Esto puede traer ciertos tropiezos de alguna manera sutil. Nuestro ejemplo anterior devolvió un vector (usando `length`), pero es interesante ver lo que sucede cuando se pasa una lista vacía:

In [23]:
sapply(list(), length)
## list()

list()

Si la lista de entrada tiene una longitud cero, entonces `sapply` siempre devuelve una lista, independientemente de la función que se pasa. Por lo tanto, si sus datos pudieran estar vacíos y conozca el valor devuelto, es más seguro utilizar `vapply`:


In [24]:
vapply(list(), length, numeric(1))
## numeric(0)

numeric(0)

#### Función vapply

Aunque `sapply` es muy práctico y elegante, algunas veces estas potencialidades pueden  convertirse en un riesgo. Supongamos que tenemos una lista de números de entrada:

In [25]:
x <- list(c(1, 2), c(2, 3), c(1, 3))

Si queremos obtener un vector numérico de los números al cuadrados para cada número en `x`, `sapply`,  puede ser fácil de usar porque automáticamente trata de simplificar la estructura de datos del resultado:

In [26]:
sapply(x, function(x) x ^ 2)

1,4,1
4,9,9


Sin embargo, si los datos de entrada tienen algunos errores, `sapply()` aceptará silenciosamente la entrada y puede devolver un valor inesperado. Por ejemplo, supongamos que el tercer elemento de `x` ha recibido por error un elemento adicional:

In [27]:
x1 <- list(c(1, 2), c(2, 3), c(1, 3, 3))

Entonces, `sapply()` encuentra que ya no se puede simplificar a una matriz y, por tanto, devuelve una lista:

In [28]:
sapply(x1, function(x) x ^ 2)

[[1]]
[1] 1 4

[[2]]
[1] 4 9

[[3]]
[1] 1 9 9

Si usamos `vapply()` en primer lugar, el error será detectado muy pronto. La función `vapply()` tiene un argumento adicional que especifica el valor devuelto de cada iteración. En el código siguiente, tiene el argumento `numeric(2)`, lo que significa que cada iteración debe devolver un vector numérico de dos elementos. Si este argumento no se cumple, la función terminará en un error:

In [29]:
vapply(x1, function(x) x ^ 2, numeric(2))

ERROR: Error in vapply(x1, function(x) x^2, numeric(2)): Los valores deben ser de longitud 2, 
pero el resultado FUN(X [[3]]) es la longitud 3 


In [30]:
# Para el ejemplo original, se cumple el resultado

vapply(x, function(x) x ^ 2, numeric(2))

1,4,1
4,9,9


En conclusión, `sapply()` y `vapply()` son muy similares a `lapply()` excepto que simplifican su salida para producir un vector atómico. Mientras que `sapply()` adivina, `vapply()` toma un argumento adicional que especifica el tipo de salida. La función `sapply()` es ideal para el uso interactivo, ya que ahorra escribir, pero si lo usas dentro de  funciones obtendrás errores extraños si proporcionas el tipo de entrada incorrecto, `vapply()`  es más detallado, pero da más mensajes informativos de error y nunca falla en silencio. Es más adecuado para su uso dentro de otras funciones.

Los siguientes ejemplos ilustran estas diferencias. Cuando se le da un data frame, `sapply()` y `vapply()` devuelven los mismos resultados. Cuando se le da una lista vacía, `sapply()` devuelve otra lista vacía en lugar del vector lógico de longitud cero  correcto.

In [31]:
sapply(mtcars, is.numeric)

mpg  cyl disp   hp drat   wt qsec   vs   am gear carb 
TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [32]:
vapply(mtcars, is.numeric, logical(1))

mpg  cyl disp   hp drat   wt qsec   vs   am gear carb 
TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [33]:
sapply(list(), is.numeric)
## list()

list()

In [34]:
vapply(list(), is.numeric, logical(1))
## logical(0)

logical(0)

El siguiente ejemplo ilustra un posible problema al extraer la clase de columnas en un data frame: si se asume falsamente que la clase sólo tiene un valor y se utiliza `sapply()`, no se notará  el problema hasta que se dé una función futura a una lista en lugar de a un vector de caracteres.

In [35]:
df <- data.frame(x = 1:10, y = letters[1:10])
sapply(df, class)

x         y 
"integer"  "factor"

In [36]:
vapply(df, class, character(1))

x         y 
"integer"  "factor"

In [37]:
df2 <- data.frame(x = 1:10, y = Sys.time() + 1:10)
sapply(df2, class)

$x
[1] "integer"

$y
[1] "POSIXct" "POSIXt"

In [38]:
vapply(df2, class, character(1))

ERROR: Error in vapply(df2, class, character(1)): Los valores deben ser de longitud 1, 
pero el resultado FUN(X [[2]]) es la longitud 2 


`sapply()` es una envoltura  alrededor de `lapply()` que transforma una lista en un vector en el paso final. `vapply()` es una implementación de `lapply()` que asigna resultados a un vector (o matriz) de tipo apropiado en lugar de  una lista. El código siguiente muestra una implementación en R de las funciones  `sapply()` y `vapply()` que tienen diferentes salidas desde `lapply`:

In [39]:
sapply2 <- function(x, f, ...) {
    res <- lapply2(x, f, ...)
    simplify2array(res)
}

vapply2 <- function(x, f, f.value, ...) {
    out <- matrix(rep(f.value, length(x)), nrow = length(f.value))
    for (i in seq_along(x)) {
        res <- f(x[[i]], ...)
        stopifnot(
            length(res) == length(f.value),
            typeof(res) == typeof(f.value)
    )
    out[ ,i] <- res
  }
  out
}

![](sapply-vapply.png)

#### Función mapply

Con `lapply()`, sólo un argumento varía en la  función, los otros argumentos son fijos. Esto hace que sea poco adecuado para algunos problemas. Por ejemplo, ¿cómo se encontraría el  promedio ponderado cuando se tiene dos listas, una de observaciones y otra de promedios?.


In [40]:
# Ejemplo de Advanced R de  Hadley Wickham

# Generamos algunos datos muestrales
xs <- replicate(5, runif(10), simplify = FALSE)
ws <- replicate(5, rpois(10, 5) + 1, simplify = FALSE)

# Usamos lapply() para calcular los medios no ponderados:

unlist(lapply(xs, mean))

[1] 0.4328599 0.6218232 0.5892702 0.4570522 0.5995637

Pero, ¿cómo podemos proporcionar los promedios a `weighted.mean()?`.  `lapply (x, means, w)` no funcionará porque los argumentos adicionales a `lapply()` se pasan a cada llamada. Podríamos cambiar la forma del bucle, que funciona, pero no es la manera adecuada. Podria usarse en su  lugar la función `Map` una variante de `lapply`.

In [41]:
unlist(lapply(seq_along(xs), function(i) {
    weighted.mean(xs[[i]], ws[[i]])
}))

[1] 0.4767248 0.6496591 0.5482914 0.5448805 0.6264120

La función `mapply`, una abreviación  para `multiple argument list apply`, permite pasar el mayor número de vectores que se desee. Un uso común es pasar en una lista en un argumento y los nombres de esa lista en otro, resolviendo los problemas que tiene `lapply`. Una pequeña molestia es que para acomodar un número arbitrario de argumentos vectoriales, el orden de los argumentos ha sido cambiado. Para `mapply`, la función se pasa como  primer argumento:

In [42]:
msg <- function(nombre, factores)
{
    ifelse(
        length(factores) == 1,
        paste(nombre, "es primo"),
        paste(nombre ," tiene factores", toString(factores))
    )
}
mapply(msg, names(primo_factores), primo_factores)

dos                           tres 
                "dos es primo"                "tres es primo" 
                        cuatro                          cinco 
 "cuatro  tiene factores 2, 2"               "cinco es primo" 
                          seis                          siete 
   "seis  tiene factores 2, 3"               "siete es primo" 
                          ocho                          nueve 
"ocho  tiene factores 2, 2, 2"   "nueve  tiene factores 3, 3" 
                          diez 
   "diez  tiene factores 2, 5"

En conclusión, mientras `lappy()` y `sapply()` iteran sobre un vector, `mapply()` itera sobre múltiples vectores. En otras palabras, `mapply` es una versión multivariable de `sapply`:

In [43]:
mapply(function(a, b, c) a * b + b * c + a * c, a = c(1, 2, 3), b = c(5, 6, 7), c = c(-1, -2, -3))

[1] -1 -4 -9

La función de iteración permite regresar no sólo valores escalares, sino vectores de elementos múltiples. Entonces, `mapply()` simplificará el resultado, al igual que `sapply()`:

In [44]:
df3 <- data.frame(v1 = c(1, 2, 3), v2 = c(3, 4, 5))
df3

v1,v2
1,3
2,4
3,5


In [45]:
mapply(function(xi, yi) c(xi, yi, xi + yi), df3$v1, df3$v2)

1,2,3
3,4,5
4,6,8


`Map` es la versión multivariable de `lapply` y por lo tanto, siempre devuelve una lista y también del ejemplo anterior:

In [46]:
Map(function(xi, yi) c(xi, yi, xi + yi), df3$v1, df3$v2)

[[1]]
[1] 1 3 4

[[2]]
[1] 2 4 6

[[3]]
[1] 3 5 8

In [47]:
unlist(Map(weighted.mean, xs, ws))

[1] 0.4767248 0.6496591 0.5482914 0.5448805 0.6264120

Tenga en cuenta que el orden de los argumentos es un poco diferente: `function` es el primer argumento para `Map()` y el segundo para `lapply()`. 

`Map` es útil siempre que tenga dos (o más) listas (o data frames) que necesite procesar en paralelo. Por ejemplo, otra forma de estandarizar las columnas es calcular primero los medios y después dividirlos. Podríamos hacer esto con `lapply()`, pero si lo hacemos en dos pasos, podemos ver más fácilmente los resultados en cada paso, lo cual es particularmente importante si el primer paso es más complicado.

In [48]:
mtmeans <- lapply(mtcars, mean)
mtmeans2 <- Map('/', mtcars, mtmeans)

# En este caso es , equivalente a 
mtcars2 <- lapply(mtcars, function(x) x / mean(x))

 Si algunos de los argumentos deben ser fijos y constantes, utilice una función anónima. La función `Map ` es equivalente a  `mapply`, con `simplify = FALSE`, que es casi siempre lo que se desea.

In [49]:
Map(function(x, w) weighted.mean(x, w, na.rm = TRUE), xs, ws)

[[1]]
[1] 0.4767248

[[2]]
[1] 0.6496591

[[3]]
[1] 0.5482914

[[4]]
[1] 0.5448805

[[5]]
[1] 0.626412

En lugar de utilizar una función anónima para proporcionar entradas constantes, `mapply` tiene el argumento `MoreArgs` que toma una lista de argumentos adicionales que se proporcionarán en cada llamada. Esto no cumple con  la semántica de evaluación habitual de R, y es inconsistente con otras funciones.

La función `Vectorize` es un envoltorio para `mapply` que toma una función que normalmente acepta una entrada escalar y devuelve una nueva función que acepta vectores. Esta función siguiente no está vectorizada debido al  uso de `switch`, que requiere una entrada escalar:

In [50]:
reporte_genero <- function(genero)
{
    switch(
        genero,
        masculino = "Es un niño !",
        femenino = "Es una niña!",
        "Um..."
    )
}

Si pasamos un vector a la función, lanzará un error:

In [51]:
genero <- c("masculino", "femenino", "otro")
reporte_genero(genero)

ERROR: Error in switch(genero, masculino = "Es un niño !", femenino = "Es una niña!", : EXPR must be a length 1 vector


Si bien es teóricamente posible hacer una reescritura completa de una función que está  vectorizada, es más fácil utilizar la función `Vectorize`:

In [52]:
vectorizada_reporte_genero <- Vectorize(reporte_genero)
vectorizada_reporte_genero(genero)

masculino       femenino           otro 
"Es un niño !" "Es una niña!"        "Um..."

### Lecturas

* [Using apply, sapply, lapply in R](http://petewerner.blogspot.pe/2012/12/using-apply-sapply-lapply-in-r.html).
* [A brief introduction to “apply” in R](https://nsaunders.wordpress.com/2010/08/20/a-brief-introduction-to-apply-in-r/).